In [2]:
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import sys 
sys.path.append("..")
import keras.backend as K
from lib.config import *
from lib.dataset import *
from lib.utils import *
import numpy as np
from PIL import Image
import imageio
from skimage.measure import compare_ssim
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

In [6]:
true_imgs_path = './true_imgs/'
pred_imgs_path = './pred_imgs/'

output_frames = 10
input_frames = 5

true_imgs_file = os.listdir(true_imgs_path)
true_imgs_file.sort()

basic_seq = []
next_seq = []
for i in range(input_frames):
    basic_data = image_initialize(true_imgs_path+true_imgs_file[i], HEIGHT, IMG_CHNS)
    basic_seq.append(basic_data)
for i in range(input_frames,input_frames+output_frames):
    next_data = image_initialize(true_imgs_path+true_imgs_file[i], HEIGHT, IMG_CHNS)
    next_seq.append(next_data)

val_basic = np.expand_dims(np.array(basic_seq),0)
val_next = np.expand_dims(np.array(next_seq),0)
    
print(val_basic.shape)
print(val_next.shape)

(1, 5, 60, 60, 1)
(1, 10, 60, 60, 1)


In [5]:
model_no = [80,93,54,96,9,48,59,49,97,98]

for k in range(output_frames):
    frame = k+1
    print('frame:', frame)
    model_path = '../models/'+str(input_frames)+'-'+str(frame)+str(model_no[k])+'h5'
    
    K.clear_session()
    model = init_model(input_frames)
    model.load_weights(model_path, by_name=True)
    gen_imgs = model.predict(val_basic[0:1])

    true_sum = np.sum(val_next[0, k, :, :, 0])
    pred_sum = np.sum(gen_imgs[0, :, :, 0])
    ssim = compare_ssim(val_next[0, k, :, :, 0], gen_imgs[0, :, :, 0], data_range=1)
    mse = mean_squared_error(val_next[0, k, :, :, 0], gen_imgs[0, :, :, 0])
    print('mse:', mse)
    print('ssim:', ssim)
    
    imageio.imwrite(pred_imgs_path+str(frame)+'.jpg', np.uint8(gen_imgs[0]*255))

frame: 1
mse: 0.00043991662
ssim: 0.8921240929500551
frame: 2
mse: 0.00060024124
ssim: 0.8726328824992509
frame: 3
mse: 0.00062221807
ssim: 0.8555467544018025
frame: 4
mse: 0.00054342824
ssim: 0.8678783021762372
frame: 5
mse: 0.000656128
ssim: 0.8426320950974998
frame: 6
mse: 0.0007971693
ssim: 0.8302440639697465
frame: 7
mse: 0.0007943871
ssim: 0.8228952120150493
frame: 8
mse: 0.00076813693
ssim: 0.7890682838361911
frame: 9
mse: 0.0009067914
ssim: 0.8053655069670855
frame: 10
mse: 0.0008047313
ssim: 0.8059594959721054
